### Use standalone verison of ICEPLUME to calculate subglacial discharge from rom

In [1]:
%matplotlib inline

# import modules
from datetime import datetime, timedelta
import subprocess
import glob
import pickle

import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc

import pyroms
from cmocean import cm

plt.rcParams['figure.figsize'] = [18, 16]
plt.rcParams['font.size'] = 20

scrip.so not found. Remapping function will not be available


In [2]:
# -------------- functionals --------------------------------
def get_zr(zeta, h, vgrid):
    """ get z at rho points from grid and zeta info. """

    ti = zeta.shape[0]
    zr = np.empty((ti, vgrid.N) + h.shape, 'd')
    if vgrid.Vtrans == 1:
        for n in range(ti):
            for k in range(vgrid.N):
                z0 = vgrid.hc * vgrid.s_rho[k] + (h - vgrid.hc) * vgrid.Cs_r[k]
                zr[n, k, :] = z0 + zeta[n, :] * (1.0 + z0 / h)
    elif vgrid.Vtrans == 2 or vgrid.Vtrans == 4 or vgrid.Vtrans == 5:
        for n in range(ti):
            for k in range(vgrid.N):
                z0 = (vgrid.hc * vgrid.s_rho[k] + h * vgrid.Cs_r[k]) / (vgrid.hc + h)
                zr[n, k, :] = zeta[n, :] + (zeta[n, :] + h) * z0

    return zr


def get_zw(zeta, h, vgrid):
    """ get z at rho points from grid and zeta info. """

    ti = zeta.shape[0]
    zw = np.empty((ti, vgrid.Np) + h.shape, 'd')
    if vgrid.Vtrans == 1:
        for n in range(ti):
            for k in range(vgrid.Np):
                z0 = vgrid.hc * vgrid.s_w[k] + (h - vgrid.hc) * vgrid.Cs_w[k]
                zw[n, k, :] = z0 + zeta[n, :] * (1.0 + z0 / h)
    elif vgrid.Vtrans == 2 or vgrid.Vtrans == 4 or vgrid.Vtrans == 5:
        for n in range(ti):
            for k in range(vgrid.Np):
                z0 = (vgrid.hc * vgrid.s_w[k] + h * vgrid.Cs_w[k]) / (vgrid.hc + h)
                zw[n, k, :] = zeta[n, :] + (zeta[n, :] + h) * z0

    return zw

In [3]:
# ------------ build the executable ------------------------------------
subprocess.call('cd ..; ./build.bash', shell=True)

0

In [7]:
# ------------ parameters -----------------------------------------------
runoff_list = [0, 5, 50, 100, 150, 250]
Ds_list = [50, 100, 200, 400, 800]
m2amp_list = [0, 1, 2 ,4, 6]

W = 2000.
Ds = 200.
m2amp = 0.00
s2amp = 0.00
dth = 50.
dthp = 0.025
runoff = 150.
runoff_sratio = 0

dx0 = 200
dy0 = 200

y1 = 131
y2 = 141

apd_list = []

if runoff_sratio == 0:
    apd_list.append('_w%05d_s%03d_r%04d_m2amp%4.2f_s2amp%4.2f_dth%03d_dthp%5.3f' \
                    % (W, Ds, runoff, m2amp, s2amp, dth, dthp))
else:
    apd_list.append('_w%05d_s%03d_r%04d_rr%4.2f_m2amp%4.2f_s2amp%4.2f_dth%03d_dthp%5.3f' \
                    % (W, Ds, runoff, runoff_sratio, m2amp, s2amp, dth, dthp))

apd_list = []

# for runoff in runoff_list:
for Ds in Ds_list:
# for m2amp in m2amp_list:
    if runoff_sratio == 0:
        apd_list.append('_w%05d_s%03d_r%04d_m2amp%4.2f_s2amp%4.2f_dth%03d_dthp%5.3f' \
                        % (W, Ds, runoff, m2amp, s2amp, dth, dthp))
    else:
        apd_list.append('_w%05d_s%03d_r%04d_rr%4.2f_m2amp%4.2f_s2amp%4.2f_dth%03d_dthp%5.3f' \
                        % (W, Ds, runoff, runoff_sratio, m2amp, s2amp, dth, dthp))

# case_list = glob.glob('/glade/scratch/chuning/tmpdir_tidal_fjord/outputs_w*')
# apd_list = []
# for casei in case_list:
#     apd_list.append(casei.split('outputs')[-1])
    
# apd_list = ['_cd']

print(apd_list)

['_w02000_s050_r0150_m2amp0.00_s2amp0.00_dth050_dthp0.025', '_w02000_s100_r0150_m2amp0.00_s2amp0.00_dth050_dthp0.025', '_w02000_s200_r0150_m2amp0.00_s2amp0.00_dth050_dthp0.025', '_w02000_s400_r0150_m2amp0.00_s2amp0.00_dth050_dthp0.025', '_w02000_s800_r0150_m2amp0.00_s2amp0.00_dth050_dthp0.025']


In [8]:
runoff_list = []
for apd in apd_list:
    if apd == '_cd':
        runoff_list.append(150)
    else:
        runoff_i = float(apd.split('_r0')[-1][:3])
        if len(apd.split('_rr'))>1:
            rr_i = float(apd.split('_rr')[-1][:3])
            runoff_i = runoff_i*(1-rr_i)
        runoff_list.append(runoff_i)

In [9]:
fname_list = []
grid_file_list = []
hist_file_list = []
fname_out_list = []

for apd in apd_list:
    fname_list.append('/glade/work/chuning/tidal_fjord/data/glacier/tidal_fjord_glacier' + apd + '.nc')
    grid_file_list.append('/glade/u/home/chuning/tidal_fjord/tidal_fjord_grid' + apd + '.nc')
    hist_file_list.append('/glade/scratch/chuning/tmpdir_tidal_fjord/outputs' + \
                          apd + '/outputs/1900/tidal_fjord' + \
                          apd + '_his_1900-01-01T00:00:00.nc')
    fname_out_list.append('/glade/work/chuning/tidal_fjord/data/iceplume/tidal_fjord_iceplume' + \
                          apd + '.pickle')

In [10]:
for ci in range(len(apd_list)):
    runoff = runoff_list[ci]
    apd = apd_list[ci]
    fname = fname_list[ci]
    grid_file = grid_file_list[ci]
    hist_file = hist_file_list[ci]
    fname_out = fname_out_list[ci]
    
    # load grid
    grd = pyroms.grid.get_ROMS_grid(apd, hist_file=hist_file, grid_file=grid_file)
    N = grd.vgrid.N
    h = grd.vgrid.h[y1:y2, 2:3]

    # load data
    fin = nc.Dataset(fname, 'r')
    time = fin.variables['ocean_time'][:]
    zeta = fin.variables['zeta'][:]
    salt = fin.variables['salt'][:]
    temp = fin.variables['temp'][:]
    dye01 = fin.variables['dye_01'][:]
    v_raw = fin.variables['v'][:]
    w = 0.5*(fin.variables['w'][:, 1:] +
             fin.variables['w'][:, :-1])
    fin.close()

    v_raw[v_raw.mask] = 0
    v = 0.5*(v_raw[:, :, 1:] + v_raw[:, :, :-1])
    
    tN = len(time)

    zw = get_zw(zeta, h, grd.vgrid)
    zr = get_zr(zeta, h, grd.vgrid)

    # ------------ initiate variables --------------------------------------
    f = open('../outputs/plume_out_zr.txt', 'rb')
    header_zr = f.readline().split()
    f.close()
    f = open('../outputs/plume_out_zw.txt', 'rb')
    header_zw = f.readline().split()
    f.close()

    for i in range(len(header_zr)):
        header_zr[i] = header_zr[i].decode('utf-8')
    for i in range(len(header_zw)):
        header_zw[i] = header_zw[i].decode('utf-8')

    data = {}
    data['time'] = time
    data['zeta'] = zeta.squeeze()
    for (i, header) in enumerate(header_zr):
        data[header] = np.zeros((tN, N, y2-y1))
    for (i, header) in enumerate(header_zw):
        data[header] = np.zeros((tN, N+1, y2-y1))

    print('Run the module')
    
    for ti in range(tN):
        for yi in range(y2-y1):
            np.savetxt('../data/iceplume_zw.txt', zw[ti, :, yi, 0])
            np.savetxt('../data/iceplume_t.txt', temp[ti, :, yi, 0])
            np.savetxt('../data/iceplume_s.txt', salt[ti, :, yi, 0])
            np.savetxt('../data/iceplume_dye01.txt', dye01[ti, :, yi, 0])
            np.savetxt('../data/iceplume_v.txt', v[ti, :, yi, 0])
            np.savetxt('../data/iceplume_w.txt', w[ti, :, yi, 0])

            if yi == 5:
                np.savetxt('../data/iceplume_qini.txt', np.array([runoff]))
            else:
                np.savetxt('../data/iceplume_qini.txt', np.array([0.0]))

            # ------------ run the executable --------------------------------------
            subprocess.call('cd ..; ./iceplume_test.exe', shell=True)

            # ------------ load results from txt files -----------------------------
            data_zr = np.loadtxt('../outputs/plume_out_zr.txt', skiprows=1)
            data_zw = np.loadtxt('../outputs/plume_out_zw.txt', skiprows=1)

            for (i, header) in enumerate(header_zr):
                if str(header) != 'lev':
                    data[header][ti, :, yi] = data_zr[:, i]
            for (i, header) in enumerate(header_zw):
                data[header][ti, :, yi] = data_zw[:, i]

    # save outputs to pickle
    fh = open(fname_out, 'wb')
    pickle.dump(data, fh)
    fh.close()

Load cartesian grid from file
Run the module
Load cartesian grid from file
Run the module
Load cartesian grid from file
Run the module
Load cartesian grid from file
Run the module
Load cartesian grid from file
Run the module
